# Weights and Biases

> A plugin that handles logging to a [Weights and Biases](https://wandb.ai/site) project.

`WandBPlugin` logs data and training information to a Weights and Biases project, making it easy to keep track of preprocessing strategies and models, as well as creating experiment reports.

In [ ]:
# | default_exp plugins.wandb

In [ ]:
# | hide


from nbdev.showdoc import *

In [ ]:
# | export


from typing import Optional
from pathlib import Path

import wandb
import joblib
import pandas as pd
import numpy as np
from plotly.graph_objs._figure import Figure
from sklearn.base import BaseEstimator
from sklearn.utils.validation import check_is_fitted
from sklearn.exceptions import NotFittedError
from sklearn.pipeline import Pipeline

from poniard.plugins.core import BasePlugin

In [ ]:
# | export


class WandBPlugin(BasePlugin):
    """Weights and Biases plugin. Kwargs from the constructor are passed to `wandb.init()`.

    Parameters
    ----------
    project :
        Name of the Weights and Biases project.
    entity :
        Name of the Weights and Biases entity (username).
    """

    def __init__(
        self, project: Optional[str] = None, entity: Optional[str] = None, **kwargs
    ):
        super().__init__()
        self.project = project
        self.entity = entity
        self.run = wandb.init(project=project, entity=entity, **kwargs)

    def build_config(self) -> dict:
        """Helper method that builds a config dict from the poniard instance."""
        try:
            estimators = list(self._poniard.estimators.values())
        except AttributeError:
            estimators = [
                x.__class__.__name__ for x in self._poniard._default_estimators
            ]
        return {
            "estimators": estimators,
            "metrics": self._poniard.metrics,
            "cv": self._poniard.cv,
            "preprocess": self._poniard.preprocess,
            "preprocessor": self._poniard.preprocessor,
            "custom_preprocessor": self._poniard.custom_preprocessor,
            "numeric_imputer": self._poniard.numeric_imputer,
            "scaler": self._poniard.scaler,
            "numeric_threshold": self._poniard.numeric_threshold,
            "cardinality_threshold": self._poniard.cardinality_threshold,
            "high_cardinality_encoder": self._poniard.high_cardinality_encoder,
            "verbosity": self._poniard.verbose,
            "n_jobs": self._poniard.n_jobs,
            "random_state": self._poniard.random_state,
        }

    def on_setup_end(self) -> None:
        """Log WandB config that contains the parameters passed
        to the Poniard estimator constructor."""
        config = self.build_config()
        wandb.config.update(config)
        return

    def on_setup_data(self) -> None:
        """Log features and target(s) as a single dataset artifact."""
        X, y = self._poniard.X, self._poniard.y
        try:
            dataset = pd.concat([X, y], axis=1)
        except TypeError:
            dataset = np.column_stack([X, y])
            dataset = pd.DataFrame(dataset)
        table = wandb.Table(dataframe=dataset)
        artifact = wandb.Artifact(name="dataset", type="dataset")
        artifact.add(table, "Dataset")
        wandb.log_artifact(artifact)
        return

    def on_infer_types(self):
        """Log the inferred types dataframe as a table."""
        table = wandb.Table(dataframe=self._poniard.inferred_types)
        wandb.log({"Inferred types": table})
        return

    def on_setup_preprocessor(self) -> None:
        """Log the preprocessor's HTML repr."""
        wandb.log(
            {"Preprocessor": wandb.Html(self._poniard.preprocessor._repr_html_())}
        )
        return

    def on_plot(self, figure: Figure, name: str):
        """Log a Plotly plot."""
        wandb.log({name: figure})
        return

    def on_fit_end(self):
        """Log the results dataframe as a table."""
        results = self._poniard.get_results().reset_index()
        results.rename(columns={"index": "Estimator"}, inplace=True)
        table = wandb.Table(dataframe=results)
        wandb.log({"Results": table})
        for test_metric in results.columns[results.columns.str.startswith("test_")]:
            aux_table = wandb.Table(dataframe=results[["Estimator", test_metric]])
            wandb.log(
                {
                    f"{test_metric} plot": wandb.plot.bar(
                        aux_table, label="Estimator", value=test_metric
                    )
                }
            )
        return

    def on_get_estimator(self, estimator: BaseEstimator, name: str):
        """Log fitted estimator as an artifact."""
        X, y = self._poniard.X, self._poniard.y
        saved_model_path = Path(wandb.run.dir, f"{name}.joblib")
        (
            X_train,
            X_test,
            y_train,
            y_test,
        ) = self._poniard._train_test_split_from_cv()
        estimator.fit(X_train, y_train)
        joblib.dump(estimator, saved_model_path)
        artifact = wandb.Artifact(name=f"saved_estimators", type="model")
        artifact.add_file(saved_model_path.as_posix(), name=name)
        wandb.log_artifact(artifact)

        wandb.log({f"{name} pipeline": wandb.Html(estimator._repr_html_())})

        # Wandb complains about missing and non-numeric features despite the estimator
        # having everything to deal with them, so we comment out each function that takes X.
        # y_pred = estimator.predict(X_test)
        # estimator_type = self._poniard.poniard_task
        # if estimator_type == "classification":
        #     labels = np.unique(y_test)
        #     wandb.sklearn.plot_confusion_matrix(y_test, y_pred, labels)
        #     if hasattr(estimator, "predict_proba"):
        #         y_probas = estimator.predict_proba(X_test)
        #         wandb.sklearn.plot_roc(y_test, y_probas, labels)
        #         wandb.sklearn.plot_precision_recall(y_test, y_probas, labels)

        # wandb.sklearn.plot_learning_curve(estimator, X_train, y_train)
        # wandb.sklearn.plot_calibration_curve(estimator, X_train, y_train, name)

        # Remove temporarily so we can figure out how to get column names.
        # if isinstance(estimator, Pipeline):
        #     estimator = estimator[-1]
        # wandb.sklearn.plot_feature_importances(estimator, pd.DataFrame(X).columns)
        # else:
        # wandb.sklearn.plot_residuals(estimator, X_train, y_train)
        # wandb.sklearn.plot_outlier_candidates(estimator, X_train, y_train)
        return

    def on_remove_estimators(self):
        """Log config and results table."""
        if hasattr(self._poniard, "_means"):
            self.on_fit_end()
        self.on_setup_end()
        return

    def on_add_estimators(self):
        """Rerun logging at setup end."""
        self.on_setup_end()
        return

    def on_add_preprocessing_step(self):
        """Rerun logging at setup end."""
        self.on_setup_end()
        self.on_setup_preprocessor()
        return

    def on_reassign_types(self):
        """Rerun logging at setup end."""
        self.on_setup_end()
        self.on_setup_preprocessor()
        return

In [ ]:
show_doc(WandBPlugin.on_setup_data)

---

[source](https://github.com/rxavier/poniard/blob/master/poniard/plugins/wandb.py#L74){target="_blank" style="float:right; font-size:smaller"}

### WandBPlugin.on_setup_data

>      WandBPlugin.on_setup_data ()

Log features and target(s) as a single dataset artifact.

In [ ]:
show_doc(WandBPlugin.on_setup_end)

---

[source](https://github.com/rxavier/poniard/blob/master/poniard/plugins/wandb.py#L67){target="_blank" style="float:right; font-size:smaller"}

### WandBPlugin.on_setup_end

>      WandBPlugin.on_setup_end ()

Log WandB config that contains the parameters passed
to the Poniard estimator constructor.

In [ ]:
show_doc(WandBPlugin.on_infer_types)

---

[source](https://github.com/rxavier/poniard/blob/master/poniard/plugins/wandb.py#L88){target="_blank" style="float:right; font-size:smaller"}

### WandBPlugin.on_infer_types

>      WandBPlugin.on_infer_types ()

Log the inferred types dataframe as a table.

In [ ]:
show_doc(WandBPlugin.on_setup_preprocessor)

---

[source](https://github.com/rxavier/poniard/blob/master/poniard/plugins/wandb.py#L94){target="_blank" style="float:right; font-size:smaller"}

### WandBPlugin.on_setup_preprocessor

>      WandBPlugin.on_setup_preprocessor ()

Log the preprocessor's HTML repr.

In [ ]:
show_doc(WandBPlugin.on_plot)

---

[source](https://github.com/rxavier/poniard/blob/master/poniard/plugins/wandb.py#L101){target="_blank" style="float:right; font-size:smaller"}

### WandBPlugin.on_plot

>      WandBPlugin.on_plot (figure:plotly.graph_objs._figure.Figure, name:str)

Log a Plotly plot.

In [ ]:
show_doc(WandBPlugin.on_fit_end)

---

[source](https://github.com/rxavier/poniard/blob/master/poniard/plugins/wandb.py#L106){target="_blank" style="float:right; font-size:smaller"}

### WandBPlugin.on_fit_end

>      WandBPlugin.on_fit_end ()

Log the results dataframe as a table.

In [ ]:
show_doc(WandBPlugin.on_get_estimator)

---

[source](https://github.com/rxavier/poniard/blob/master/poniard/plugins/wandb.py#L123){target="_blank" style="float:right; font-size:smaller"}

### WandBPlugin.on_get_estimator

>      WandBPlugin.on_get_estimator (estimator:sklearn.base.BaseEstimator,
>                                    name:str)

Log fitted estimator as an artifact.

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()